# Understanding the MelSpectrogram

There are five control parameters that control how the spectrogram is generated. In this notebook we created an interactive interface that allows you to play with the values and explore how they effect the spectrogram.

Currently the code visualizes a dog barking sound from the Cats-vs-Dogs data set.  To change the file change the variable `MainFile` to point towards the audio file you want to use.(Tab completion works mostly for folders)

In [3]:
from __future__ import print_function

%matplotlib inline
import cv2
import csv
import numpy as np
import librosa
import wave
import os
import matplotlib.pyplot as plt

from ipywidgets import interactive
import ipywidgets as widgets

from PIL import Image
import IPython.display as displayImg

import pickle
from ipywidgets import interact, widgets
from IPython.display import Image, display, HTML
import glob

In [15]:
##SpectrographVariables
SpectrumVariables={}
SpectrumVariables["RESOLUTION"] =0
SpectrumVariables["SAMPLE_RATE"]=0
SpectrumVariables["N_FFT"]=0
SpectrumVariables["HOP_LENGTH"]=0
SpectrumVariables["FMIN"]=0
SpectrumVariables["FMAX"]=0

The following cell defines the filename where the sample audio file should be read from. 

In [28]:
SOURCE_DATA_ROOT='../AudioData/'

def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))
style = {'description_width': 'initial'}

SourceFolderDropdown= widgets.Dropdown(options=listdir_nohidden(SOURCE_DATA_ROOT), description='Source for Training Data:',style=style)

display(SourceFolderDropdown)



Dropdown(description='Source for Training Data:', options=('../AudioData/Cats-Vs-Dogs', '../AudioData/Cats-Vs-…

In [43]:
SOURCE_FOLDER_TRAIN= SourceFolderDropdown.value
AudioFolderDropdown= widgets.Dropdown(options=listdir_nohidden(SOURCE_FOLDER_TRAIN), description='Audio Folder:',style=style)
display(AudioFolderDropdown)

Dropdown(description='Audio Folder:', options=('../AudioData/UrbanSound/siren', '../AudioData/UrbanSound/car_h…

In [44]:

AudioFileDropdown= widgets.Dropdown(options=[f for f in listdir_nohidden(AudioFolderDropdown.value) if  (f.endswith(".aif") or f.endswith(".flac") or f.endswith(".wav") or f.endswith(".m4a") or f.endswith(".mp3"))], description='Audio File:',style=style)
display(AudioFileDropdown)

Dropdown(description='Audio File:', options=('../AudioData/UrbanSound/siren/184623.wav', '../AudioData/UrbanSo…

In [45]:
import IPython.display as ipd
x,sr= librosa.load(AudioFileDropdown.value,mono=True)
ipd.Audio(x,rate=sr)

In [48]:
### Image output resolution
RESOLUTION = 224
#MainFile="../AudioData/Cats-Vs-Dogs/Dogs/365089__justiiiiin__dogs-barking.wav"
#MainFile="../AudioData/audio-cats-and-dogs/cats_dogs/train/cat/cat_23.wav"
#MainFile="../AudioData/Cats-Vs-Dogs/Cats/120017__kklab5050__cat-meow.wav"
MainFile=AudioFileDropdown.value

def f(SAMPLE_RATE, N_FFT,HOP_LENGTH,FMIN, FMAX,N_MELS,POWER):
    
    SpectrumVariables["RESOLUTION"] =RESOLUTION
    SpectrumVariables["SAMPLE_RATE"]=SAMPLE_RATE
    SpectrumVariables["N_FFT"]=N_FFT
    SpectrumVariables["HOP_LENGTH"]=HOP_LENGTH
    SpectrumVariables["FMIN"]=FMIN
    SpectrumVariables["FMAX"]=FMAX
    SpectrumVariables["N_MELS"]=N_MELS
    SpectrumVariables["POWER"]=POWER
   
    x ,sample_rate_in = librosa.load(MainFile,mono=True)
    audio_data = librosa.resample(x, sample_rate_in, SAMPLE_RATE)
    mel_spec_power = librosa.feature.melspectrogram(audio_data, sr=SAMPLE_RATE, n_fft=N_FFT, 
                                                    hop_length=HOP_LENGTH, 
                                                    n_mels=N_MELS, power=POWER,
                                                   fmin=FMIN,fmax=FMAX)
    mel_spec_db = np.float32(librosa.power_to_db(mel_spec_power, ref=np.max))
    image = mel_spec_db; # convert to float
    image -= image.min() # ensure the minimal value is 0.0
    image /= image.max() # maximum value in image is now 1.0
    image*=256;
    img = image.astype(np.uint8)
    print(img.min(),img.max())
    colerPic = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
    fig, axs = plt.subplots(2,5, figsize=(15, 6), facecolor='w', edgecolor='k')
    axs = axs.ravel()    
    for i in range(int(np.floor(colerPic.shape[1]/RESOLUTION))):
        if(i>=len(axs)):
            print("There where more pictures generated but I only could show these 10.")
            break;
        startx=RESOLUTION*i
        stopx=RESOLUTION*(i+1)
        OutputImage = cv2.resize(colerPic[:,startx:stopx,:],(RESOLUTION,RESOLUTION))
        axs[i].imshow(OutputImage)
        

interactive_plot = interactive(f, {"manual":True},SAMPLE_RATE=(16000, 48000,100), N_FFT=(128, 2048*2, 64),HOP_LENGTH=(128, 512, 16),FMIN=(0,300,5),FMAX=(300,40000,10),N_MELS=(10,300,10),POWER=(0.1,3))
interactive_plot

interactive(children=(IntSlider(value=32000, description='SAMPLE_RATE', max=48000, min=16000, step=100), IntSl…

## Saving new settings

Normally, you can copy and paste the new values by hand. For this particular workshop, we create a dictionary called ``SpectographVariables`` that contains all relevant information. 

You can load in this dictionary when you generate the spectrums or when you run the inference from your microphone. Run the box below to save the variables to a file. Remember, the file name as you will need it for the generating the spectrograms for all audio files. 


In [ ]:
fileNamed = False
while(not fileNamed):
    FileName =input("Type in your filename here => ")
    print("It will end up in the same folder as this notebook.")
    if (not os.path.exists(FileName+'.SpecVar')):
        pickle.dump(SpectrumVariables, open( FileName+'.SpecVar', "wb" ))
        print("File created as: "+FileName+".SpecVar")
        fileNamed = True
    else:
        print(FileName+".SpecVar already exists. Pick a unique name.")